# CECS 229 Coding Assignment #6

#### Due Date: 

Sunday, 4/24 @ 11:59 PM

#### Submission Instructions:

To receive credit for this assignment you must submit the following by the due date:

1. **To the BB Dropbox Folder:** this completed .ipynb file

2. **To CodePost:** this file converted to a Python script named `ca6.py`

#### Objectives:

1. Apply Gaussian Elimination, with and without pivoting, to help solve the system $A \overrightarrow{x} = \overrightarrow{b}$.
2. Use Lp -norm to calculate the error in a solution given by applying any of the Gaussian elimination algorithms.
3. Use the RREF of the augmented matrix for the system $A \overrightarrow{x} = \overrightarrow{b}$ to determine if it has one solution, no solution, or infinitely-many solutions.
4. Determine the number of free variables that the system $A \overrightarrow{x} = \overrightarrow{b}$ has if it has infinitely-many solutions.
5. Determine whether a set of column-vectors is linearly dependent by forming and solving a system of the form $A \overrightarrow{x} = \overrightarrow{0}$.
-----------------------------------------



#### Problem 1

Copy-paste your implemented `Matrix` and `Vec` classes to the next cell.  Then, complete the following tasks:

1. Add a method `norm(p)` to your `Vec` class so that if `u` is a `Vec` object, then `u.norm(p)` returns the $L_p$-norm of vector `u`.  Recall that the $L_p$-norm of an $n$-dimensional vector $\overrightarrow{u}$ is given by, $||u||_p = \left( \sum_{i = 1}^{n} |u_i|^p\right)^{1/p}$.  Input `p` should be of the type `int`.  The output norm should be of the type `float`.
2. Add a method `rank()` to your `Matrix` class so that if `A` is a `Matrix` object, then `A.rank()` returns the rank of `A`.

In [1]:
class Vec:
    def __init__(self, contents = []):
       self.elements = contents
 
    def __abs__(self):
        v = 0
        for i in self.elements:
            v += i**2
        return (math.sqrt(v))
 
    def __add__(self, other):
        #return Vec(self.elements + other)
        if(len(self.elements) == len(other.elements)):
            T = [None]*len(self.elements)
            for i in range(0,len(self.elements)):
                T[i] = self.elements[i] + other.elements[i]
        else:
            raise ValueError
        return Vec(T)
    def __sub__(self, other):
        if(len(self.elements) == len(other.elements)):
            T = [None]*len(self.elements)
            for i in range(0,len(self.elements)):
                T[i] = self.elements[i] - other.elements[i]
        else:
            raise ValueError
        return Vec(T)
 
    def __mul__(self, other):
        if type(other) == Vec:
            m = 0
            if(len(self.elements) == len(other.elements)):
                for i in range(0,len(self.elements)):
                    m += self.elements[i] * other.elements[i]
                return m
            else:
                raise ValueError        
        elif type(other) == float or type(other) == int:
            m = [None]*len(self.elements)
            for i in range(0,len(self.elements)):
                m[i] = self.elements[i]*other
        return Vec(m)
 
    def __rmul__(self, other):
        m = [None]*len(self.elements)
        for i in range(0,len(self.elements)):
            m[i] = self.elements[i]*other
        return Vec(m)

    def norm(self, p):
        if not(isinstance(p, int)):
            raise ValueError
        n = 0.00
        for i in range(0,len(self.elements)):
            n += abs(self.elements[i])**p
        return n**(1/p)
 
    def __str__(self):
        return str(self.elements)
 

 
class Matrix:
    
    def __init__(self):  
        self.colsp = []
        self.rowsp = []
 
    def __init__(self, rows):
        self.rowsp = rows
        self.colsp = [[None for i in range (0, len(self.rowsp))] for j in range(0, len(self.rowsp[0]))]
        for i in range(0, len(self.rowsp)):
            for j in range(0, len(self.rowsp[0])):
                self.colsp[j][i] = self.rowsp[i][j]
 
    def __str__(self):
        s = ""
        for i in range(len(self.rowsp)):
            for j in range(len(self.rowsp[0])):
                if j==0:
                    s += "["+str(self.rowsp[i][j])+", "
                elif j==len(self.rowsp[0])-1:
                    s += str(self.rowsp[i][j])+"]"
                else:
                    s += str(self.rowsp[i][j])+", "
            s+="\n"
        return s 

    def set_col(self, j, u):
        j -= 1
        
        if len(self.colsp[j]) != len(u):
            raise ValueError("Incompatible column length.")
        
        self.colsp[j] = u
        
        for i in range(len(self.rowsp)):
            self.rowsp[i][j] = self.colsp[j][i]
    
    def set_row(self, i, v):
        i -= 1
        
        if len(self.rowsp[i]) != len(v):
            raise ValueError("Incompatible row length.")
        
        self.rowsp[i] = v
        
        for j in range(len(self.colsp)):
            self.colsp[j][i] = self.rowsp[i][j]
 
    def set_entry(self, i, j, x):
        self.rowsp[i-1][j-1] = x
        self.colsp[j-1][i-1] = x
 
    def get_col(self, j):
        return self.colsp[j-1]
 
    def get_row(self, i):
        return self.rowsp[i-1]
 
    def get_entry(self, i, j):
        return self.rowsp[i][j]
    
    def col_space(self):
        return self.colsp
 
    def row_space(self):
        return self.rowsp
    
    def get_diag(self, k):
        diagonal = []
        k2 = 0
        matrix = self.rowsp
        length = len(matrix)
 
        if(k == 0):
            while k < length:
                diagonal.append(matrix[k][k])
                k += 1
        elif k > 0:
            while k < length:
                diagonal.append(matrix[k2][k])
                k2 += 1
                k += 1
        else:
            k = abs(k)
            while k < length:
                diagonal.append(matrix[k][k2])
                k2 += 1
                k += 1
 
        return diagonal
 
    def __add__(self, other):
        if not (isinstance(other, Matrix)):
            return ("Not of the same type.")
        
        if(len(self.rowsp) == len(other.rowsp) and len(self.colsp) == len(other.colsp)):
            result = [[None for i in range (0, len(self.colsp))] for j in range(0, len(self.colsp[0]))]
            for i in range (0, len(self.rowsp)):
                for j in range (0, len(self.colsp)):
                    result[i][j] = self.rowsp[i][j] + other.rowsp[i][j]
            M = Matrix(result)
            return M
        else:
            raise ValueError
        
    
    def __sub__(self, other):
        if not (isinstance(other, Matrix)):
            return ("Not of the same type.")
        
        if(len(self.rowsp) == len(other.rowsp) and len(self.colsp) == len(other.colsp)):
            result = [[None for i in range (0, len(self.colsp))] for j in range(0, len(self.colsp[0]))]
            for i in range (0, len(self.rowsp)):
                for j in range (0, len(self.colsp)):
                    result[i][j] = self.rowsp[i][j] - other.rowsp[i][j]
            M = Matrix(result)
            return M
        else:
            raise ValueError
        
        
    def __mul__(self, other):  
        if type(other) == float or type(other) == int:
            result = [[None for i in range (0, len(self.colsp))] for j in range(0, len(self.colsp[0]))]
            for i in range(0, len(self.rowsp)):
                for j in range(0, len(self.rowsp[0])):
                    result[i][j] = self.rowsp[i][j] * other
            M = Matrix(result)
            return M

        elif type(other) == Matrix:
            if(len(self.colsp) == len(other.rowsp)): 
                if(len(self.rowsp) > len(other.colsp)):
                    result = [[0 for i in range (0, len(self.colsp))] for j in range(0, len(other.rowsp))]
                else:
                    result = [[0 for i in range (0, len(other.colsp))] for j in range(0, len(self.rowsp))]
                for i in range(0, len(self.rowsp)):
                    for j in range(0, len(other.colsp)):
                        for k in range(0, len(self.colsp)):
                            result[i][j] += self.rowsp[i][k] * other.colsp[j][k]
                M = Matrix(result)
                return M
            else:
                raise ValueError
 
        elif type(other) == Vec:
            out = []
            for i in self.rowsp:
                out.append(other * Vec(i))
            return Vec(out)       
    
    def __rmul__(self, other):  
        if type(other) == float or type(other) == int:
            result = [[None for i in range (0, len(self.colsp))] for j in range(0, len(self.colsp[0]))]
            for i in range(0, len(self.rowsp)):
                for j in range(0, len(self.rowsp[0])):
                    result[i][j] = other * self.rowsp[i][j]
            M = Matrix(result)
            return M
        else:
            print("ERROR: Unsupported Type.")
            raise ValueError
        
    def __eq__(self, other):
        """overloads the == operator to return True if 
        two Matrix objects have the same row space and column space"""
        this_rows = self.row_space()
        other_rows = other.row_space()
        this_cols = self.col_space()
        other_cols = other.col_space()
        return this_rows == other_rows and this_cols == other_cols
 
    def __req__(self, other):
        """overloads the == operator to return True if 
        two Matrix objects have the same row space and column space"""
        this_rows = self.row_space()
        other_rows = other.row_space()
        this_cols = self.col_space()
        other_cols = other.col_space()

    def rank(self):
        rank = 0
        z = 0
        k = []
        for i in range(len(self.colsp[0])):
            k.append(0)
        b = Vec(k)

        m = Matrix(self.rowsp)
        for i in range(len(b.elements)):
            m.rowsp[i].append(b.elements[i])

        # swap if [0][0] is 0
        if m.colsp[0][0] == 0:
            c0 = Vec(m.colsp[0])
            largest_i = 0
            largest = 0
            for j in range (0, len(c0.elements)):
                if abs(c0.elements[j]) > largest:
                    largest = c0.elements[j]
                    largest_i = j
            if largest_i != 0:
                swaprow = m.rowsp[largest_i]
                m.rowsp[largest_i] = m.rowsp[0]
                m.rowsp[0] = swaprow

            m = Matrix(m.rowsp)

        # start gaussian elimination
        numRows = len(m.row_space())
        numCols = len(m.col_space())
        iterate = 0
        if numRows < numCols:
            iterate = len(m.row_space()) - 1
        else:
            iterate = len(m.col_space()) - 1
        for p in range(iterate):
            if m.col_space()[p][p] == 0:
                a = m.row_space()[p]
                b = m.row_space()[-1]
                # print(m.get_row(p))
                m.set_row(p + 1, b)
                m.set_row(numRows, a)
                # print(m.get_row(p))
            pivot = m.col_space()[p][p]
            # print("PIVOT:", pivot)
            if pivot != 0:
                for i in range(p + 1, len(m.row_space())):
                    first = m.row_space()[i][p]
                    for j in range(len(m.row_space()[0])):
                        top = m.row_space()[p][j]
                        curr = m.row_space()[i][j]
                        # print("first:", first, "top:", top, " curr:", curr)
                        m.row_space()[i][j] = curr - ((first / pivot) * top)
            m = Matrix(m.row_space())
        
        for i in range(len(m.rowsp)):
            for j in range(len(m.rowsp[0])):
                if i == j:
                    if m.rowsp[i][j] == 0:
                        for k in range(len(m.rowsp[0])):
                            if m.rowsp[i][k] == 0:
                                z += 1
                    else:
                        rank += 1
                    if z == len(m.rowsp[0]):
                        rank -= 1
                        z = 0
        return rank

In [2]:
"""RANK TESTER CELL"""
A = Matrix([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
print(A)
print("Rank:", A.rank(), "\nExpected: 2\n")

B = Matrix([[1, 2], [-1, -2]])
print(B)
print("Rank:", B.rank(), "\nExpected: 1\n")

C = Matrix([[0, -1, 5], [2, 4, -6], [1, 1, 5]])
print(C)
print("Rank:", C.rank(), "\nExpected: 3\n")

D = Matrix([[5, 3, 0], [1, 2, -4], [-2, -4, 8]])
print(D)
print("Rank:", D.rank(), "\nExpected: 2\n")

E = Matrix([[1, 2, -1, 3], [2, 4, 1, -2], [3, 6, 3, -7]])
print(E)
print("Rank:", E.rank(), "\nExpected: 2")

[0, 1, 2]
[3, 4, 5]
[6, 7, 8]

Rank: 1 
Expected: 2

[1, 2]
[-1, -2]

Rank: 0 
Expected: 1

[0, -1, 5]
[2, 4, -6]
[1, 1, 5]

Rank: 3 
Expected: 3

[5, 3, 0]
[1, 2, -4]
[-2, -4, 8]

Rank: 1 
Expected: 2

[1, 2, -1, 3]
[2, 4, 1, -2]
[3, 6, 3, -7]

Rank: 2 
Expected: 2


#### Problem 2

1. Implement a helper function called `_rref(A, b)` that applies Gaussian Elimination ***without pivoting*** to return the Reduced-Row Echelon Form of the augmented matrix formed from `Matrix` object `A` and `Vec` object `b`.  The output must be of the type `Matrix`.
2. Implement the function `solve_np(A, b)` that uses `_rref(A)` to solve the system $A \overrightarrow{x} = \overrightarrow{b}$.  The input `A` is of the type `Matrix` and `b` is of the type `Vec`.
    - If the system has a unique solution, it returns the solution as a `Vec` object.  
    - If the system has no solution, it returns `None`. 
    - If the system has infinitely many solutions, it returns the number of free variables (`int`) in the solution.

In [ ]:
def _rref(A, b):
    # build augmented matrix
    lastCol = len(A.col_space()) + 1
    rows = A.row_space()
    for i in rows:
        i.append(0)
    m = Matrix(rows)
    m.set_col(lastCol, b.elements)

    # swap if [0][0] is 0
    if m.colsp[0][0] == 0:
        c0 = Vec(m.colsp[0])
        largest_i = 0
        largest = 0
        for j in range (0, len(c0.elements)):
            if abs(c0.elements[j]) > largest:
                largest = c0.elements[j]
                largest_i = j
        if largest_i != 0:
            swaprow = m.rowsp[largest_i]
            m.rowsp[largest_i] = m.rowsp[0]
            m.rowsp[0] = swaprow

        m = Matrix(m.rowsp)

    # start gaussian elimination
    numRows = len(m.row_space())
    numCols = len(m.col_space())
    iterate = 0
    if numRows < numCols:
        iterate = len(m.row_space()) - 1
    else:
        iterate = len(m.col_space()) - 1
    for p in range(iterate):

        if m.col_space()[p][p] == 0:
            a = m.row_space()[p]
            b = m.row_space()[-1]
            # print(m.get_row(p))
            m.set_row(p + 1, b)
            m.set_row(numRows, a)
            # print(m.get_row(p))
        pivot = m.col_space()[p][p]
        # print("PIVOT:", pivot)
        if pivot != 0:
            for i in range(p + 1, len(m.row_space())):
                first = m.row_space()[i][p]
                for j in range(len(m.row_space()[0])):
                    top = m.row_space()[p][j]
                    curr = m.row_space()[i][j]
                    # print("first:", first, "top:", top, " curr:", curr)
                    m.row_space()[i][j] = curr - ((first / pivot) * top)
        m = Matrix(m.row_space())
    return Matrix(m.row_space())

def solve_np(A,b):
    m = A.rank()
    print(m)
    Ag = _rref(A,b)
    rank = Ag.rank()
    print(rank)
    sol = Vec(m.colsp[len(m.rowsp[0])-1])
    for i  in range(len(sol.elements)-1, -1, -1):
        sol.elements[i] = m.rowsp[i][len(sol.elements)] / m.rowsp[i][i]
        # print(Ag.rowsp[i])
    # a = Vec(Ag.colsp[len(Ag.rowsp[0])-2])
    return None #sol

In [ ]:
"""TESTER CELL #1 FOR GENP"""
A = Matrix([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
b = Vec.Vec([9, 10, 11])
sol = solve_np(A, b)
print("Result:", sol)
print("Expected: 1")

In [ ]:
"""TESTER CELL #2 FOR GENP"""
A = Matrix([[1, 0, 5], [0, 1, 2], [3, 2, 0]])
b = Vec([6, 3, 5])
sol = solve_np(A, b)
print("Returned:", sol)
print("Expected: [1.0, 1.0, 1.0]")

In [ ]:
"""TESTER CELL #3 FOR GENP"""
A = Matrix([[1, 1, 5], [2, 2, 10]])
b = Vec([6, 3])
sol = solve_np(A, b)
print("Returned:", sol)
print("Expected: None")

#### Problem 3

1. Implement a helper function called `_rref_pp(A, b)` that applies Gaussian Elimination ***with partial pivoting*** to return the Reduced-Row Echelon Form of the augmented matrix formed from `Matrix` object `A` and `Vec` object `b`.  The output must be of the type `Matrix`.
2. Implement the function `solve_pp(A, b)` that uses `_rref_pp(A, b)` to solve the system $A \overrightarrow{x} = \overrightarrow{b}$.  The input `A` is of the type `Matrix` and `b` is of the type `Vec`.  
    - If the system has a unique solution, it returns the solution as a `Vec` object.  
    - If the system has no solution, it returns `None`. 
    - If the system has infinitely many solutions, it returns the number of free variables (`int`) in the solution.

In [ ]:
def _rref_pp(A, b):
    # build augmented matrix
    lastCol = len(A.col_space()) + 1
    rows = A.row_space()
    for i in rows:
        i.append(0)
    m = Matrix(rows)
    m.set_col(lastCol, b.elements)

 # start gaussian elimination
    numRows = len(m.row_space())
    numCols = len(m.col_space())
    iterate = 0
    if numRows < numCols:
        iterate = len(m.row_space()) - 1
    else:
        iterate = len(m.col_space()) - 1
    for p in range(iterate):
        # c0 = Vec(m.colsp[0])
        # largest_i = 0
        # largest = 0
        # for j in range (p, len(c0.elements)):
        #     if abs(c0.elements[j]) > largest:
        #         largest = c0.elements[j]
        #         largest_i = j
        # if largest_i != 0:
        #     swaprow = m.rowsp[largest_i]
        #     m.rowsp[largest_i] = m.rowsp[p]
        #     m.rowsp[p] = swaprow

        m = Matrix(m.rowsp)    
        if m.col_space()[p][p] == 0:
            a = m.row_space()[p]
            b = m.row_space()[-1]
            m.set_row(p + 1, b)
            m.set_row(numRows, a)
        pivot = m.col_space()[p][p]
        if pivot != 0:
            for i in range(p + 1, len(m.row_space())):
                first = m.row_space()[i][p]
                for j in range(len(m.row_space()[0])):
                    top = m.row_space()[p][j]
                    curr = m.row_space()[i][j]
                    m.row_space()[i][j] = curr - ((first / pivot) * top)
        m = Matrix(m.row_space())
    return Matrix(m.row_space())

def solve_pp(A, b):
    #todo
    pass

In [ ]:
"""TESTER CELL #1 FOR GEPP"""
A = Matrix([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
b = Vec.Vec([9, 10, 11])
sol = solve_pp(A, b)
print("Result:", sol)
print("Expected: 1")

In [ ]:
"""TESTER CELL #2 FOR GEPP"""
A = Matrix([[1, 0, 5], [0, 1, 2], [3, 2, 0]])
b = Vec([6, 3, 5])
sol = solve_pp(A, b)
print("Returned:", sol)
print("Expected: [1.0000000000000002, 0.9999999999999996, 1.0000000000000002]") # or something close to this

In [ ]:
"""TESTER CELL #3 FOR GEPP"""
A = Matrix([[1, 1, 5], [2, 2, 10]])
b = Vec([6, 3])
sol = solve_pp(A, b)
print("Returned:", sol)
print("Expected: None")

#### Problem 4

1. Implement a helper function called `_rref_tp(A, b)` that applies Gaussian Elimination ***with total pivoting*** to return the Reduced-Row Echelon Form of the augmented matrix formed from `Matrix` object `A` and `Vec` object `b`.  The output must be of the type `Matrix`. 
2. Implement the function `solve_tp(A, b)` that uses `_rref_tp(A)` to solve the system $A \overrightarrow{x} = \overrightarrow{b}$.  The input `A` is of the type `Matrix` and `b` is of the type `Vec`. 
    - If the system has a unique solution, it returns the solution as a `Vec` object.  
    - If the system has no solution, it returns `None`. 
    - If the system has infinitely many solutions, it returns the number of free variables (`int`) in the solution.

In [ ]:
def _rref_tp(A, b):
    # build augmented matrix
    lastCol = len(A.col_space()) + 1
    rows = A.row_space()
    for i in rows:
        i.append(0)
    m = Matrix(rows)
    m.set_col(lastCol, b.elements)

 # start gaussian elimination
    numRows = len(m.row_space())
    numCols = len(m.col_space())
    iterate = 0
    if numRows < numCols:
        iterate = len(m.row_space()) - 1
    else:
        iterate = len(m.col_space()) - 1
    for p in range(iterate):
        # c0 = Vec(m.colsp[0])
        # largest_i = 0
        # largest = 0
        # for j in range (p, len(c0.elements)):
        #     if abs(c0.elements[j]) > largest:
        #         largest = c0.elements[j]
        #         largest_i = j
        # if largest_i != 0:
        #     swaprow = m.rowsp[largest_i]
        #     m.rowsp[largest_i] = m.rowsp[p]
        #     m.rowsp[p] = swaprow

        m = Matrix(m.rowsp)    
        if m.col_space()[p][p] == 0:
            a = m.row_space()[p]
            b = m.row_space()[-1]
            m.set_row(p + 1, b)
            m.set_row(numRows, a)
        pivot = m.col_space()[p][p]
        if pivot != 0:
            for i in range(p + 1, len(m.row_space())):
                first = m.row_space()[i][p]
                for j in range(len(m.row_space()[0])):
                    top = m.row_space()[p][j]
                    curr = m.row_space()[i][j]
                    m.row_space()[i][j] = curr - ((first / pivot) * top)
        m = Matrix(m.row_space())
    return Matrix(m.row_space())


def solve_tp(A, b):
    #todo
    pass

In [ ]:
"""TESTER CELL #1 FOR GETP"""
A = Matrix([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
b = Vec.Vec([9, 10, 11])
sol = solve_tp(A, b)
print("Result:", sol)
print("Expected: 1")

In [ ]:
"""TESTER CELL #2 FOR GETP"""
A = Matrix([[1, 0, 5], [0, 1, 2], [3, 2, 0]])
b = Vec([6, 3, 5])
sol = solve_pp(A, b)
print("Returned:", sol)
print("Expected: [1.0000000000000002, 0.9999999999999997, 1.0]") # or something close to this

In [ ]:
"""TESTER CELL #3 FOR GETP"""
A = Matrix([[1, 1, 5], [2, 2, 10]])
b = Vec([6, 3])
sol = solve_tp(A, b)
print("Returned:", sol)
print("Expected: None")

#### Master function

The following function is the master function that will be called by the CodePost tester.  It will be fully functional once you have completed Problems 1 - 4.  No edits are necessary.

In [ ]:
import enum

class GaussSolvers(enum.Enum):
    np = 0
    pp = 1
    tp = 2
    
    
def solve(A, b, solver = GaussSolvers.np):
    if solver == GaussSolvers.np:
        return solve_np(A, b)
    elif solver == GaussSolvers.pp:
        return solve_pp(A, b)
    elif solver == GaussSolvers.tp:
        return solve_tp(A, b)

In [ ]:
"""MASTER FUNCTION TESTER CELL #1"""
A = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

x = Vec([2.3, -4.1, 5.7]) # this is the true solution

b = A * x

x_np = solve(A, b)
x_pp = solve(A, b, GaussSolvers.pp)
x_tp = solve(A, b, GaussSolvers.tp)


print("No pivoting:", x_np)
print("Expected: 1")
print("Partial Pivoting:", x_pp)
print("Expected: 1")
print("Total Pivotitng:", x_tp)
print("Expected: 1")

In [ ]:
"""MASTER FUNCTION TESTER CELL #2"""
A = Matrix([[-2, 8, -3], [4, 5, 4], [-3, -1, 2]])
b = Vec([100, 75, 250])

x = Vec([-55.090497737556575, 17.98642533936652, 51.35746606334843])
x_np = solve(A, b, GaussSolvers.np)
x_pp = solve(A, b, GaussSolvers.pp)
x_tp = solve(A, b, GaussSolvers.tp)


epsilon_np = x_np - x
epsilon_pp = x_pp - x
epsilon_tp = x_tp - x


error1_np = epsilon_np.norm(1)
error2_np = epsilon_np.norm(2)

print("-"*20)
print("No Pivoting Solution:", x_np)
print("True solution:", x)

print("Errors in Gaussian Elimination Without Pivoting")
print("L1-norm error:", error1_np)
print("L2-norm error:", error2_np)
print()

print("-"*20)
print("Partial Pivoting Solution:", x_pp)
print("True solution:", x)

error1_pp = epsilon_pp.norm(1)
error2_pp = epsilon_pp.norm(2)

print("Errors in Gaussian Elimination With Partial Pivoting")
print("L1-norm error:", error1_pp)
print("L2-norm error:", error2_pp)
print()

print("-"*20)
print("Total Pivoting Solution:", x_tp)
print("True solution:", x)

error1_tp = epsilon_tp.norm(1)
error2_tp = epsilon_tp.norm(2)

print("Errors in Gaussian Elimination With Total Pivoting")
print("L1-norm error:", error1_tp)
print("L2-norm error:", error2_tp)


In [ ]:
"""MASTER FUNCTION TESTER CELL #3"""

A = Matrix([[1, 2, 3], [2, 4, 6]])

b = Vec([6, -12]) 

x_np = solve(A, b)
x_pp = solve(A, b, GaussSolvers.pp)
x_tp = solve(A, b, GaussSolvers.tp)

print("-"*20)
print("No Pivoting Solution:", x_np)
print("Expected: None")

print("-"*20)
print("Partial Pivoting Solution:", x_pp)
print("Expected: None")

print("-"*20)
print("Total Pivoting Solution:", x_tp)
print("Expected: None")


------------------------------------------------

#### Problem 5

Implement the method `is_independent(S)` that returns `True` if the set `S` of `Vec` objects is linearly **independent**, otherwise returns `False`.

In [ ]:
def is_independent(S):
    # linearly independent vectors iff the rank is equal to the number of vectors
    # create a Matrix
    vec_list = list(S)
    Ag = []
    for i in range(len(vec_list)):
        temp_list = []
        temp_list.append(vec_list[i])
        Ag.append(temp_list)

    # rref
    for i in range(len(Matrix(Ag).colsp[0]) - 1):
        if Matrix(Ag).colsp[i][i] != 0:
            for j in range(len(Matrix(Ag).colsp[0]) - 1 - i):
                r0 = Matrix(Ag).rowsp[i]
                pivot = -(Matrix(Ag).colsp[i][1 + i +
                                              j]) / Matrix(Ag).colsp[i][i]
                r = Vec(Ag.rowsp[i + 1 + j])
                r = r0 * pivot + r
                Ag.set_row(2 + i + j, r.elements)

    # find rowsp / basis

    # if dimension of the basis of the set equals the dimension of the set, the set is linearly independent

    return Matrix(Ag)

In [ ]:
"""IS-INDEPENDENT TESTER CELL"""

S1 = {Vec([1, 2]), Vec([2, 3]), Vec([3, 4])}

print("S1 is Independent:", is_independent(S1))
print("Expected: False")

S2 = {Vec([1, 1, 1]), Vec([1, 2, 3]), Vec([1, 3, 6])}

print("S2 is Independent:", is_independent(S2))
print("Expected: True")

